# Machine Learning

## Modulos a emplear

In [1]:
# Importamos las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures, scale, StandardScaler
from sklearn.decomposition import PCA
import sys
import os

# Funciones importadas como modulos
notebook_dir = os.getcwd()
scripts_dir = os.path.join(notebook_dir, "..", "src")
sys.path.append(scripts_dir)
from data_transform.print_unique_values import print_unique_values

# Codigo para imprimir mas de una salida de la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Carga y selección de la Información

### Seleccionamos la informacion del pozo clave y la editamos

In [2]:
# Definimos la ruta de la informacion procesada
file_processed = "cpe_eval.csv"
path_data_processed = os.path.join(notebook_dir, "..", "data", "processed", file_processed)

# Cargamos la información procesada
cpe_df = pd.read_csv(path_data_processed)

# Mostramos los pozos en el DataFrame
print_unique_values(cpe_df, "wellname")

Valores únicos en la columna 'wellname': Caparroso-71, Pijije-101, Pijije-111, Pijije-31REE, Pijije-123, Pijije-123REE, Pijije-125, Pijije-133, Pijije-2, Pijije-23, Pijije-25, Pijije-25REE2, Pijije-27, Pijije-34


### Seleccionamos el pozo/pozos de interes para entrenar el modelo

In [4]:
# Seleccionamos el pozo/los pozos de interes
key_wells_lst = ["Pijije-111"]
# Creamos el DataFrame key_well con la informacion de interes
key_well = cpe_df[cpe_df["wellname"].isin(key_wells_lst)].copy()

# Definimos el pozo a predecir
well_target = "Pijije-101"

# Definos las cimas y registros para predecir
key_well_tops = ["cretacico", "jurasico"]
features = ["gr", "rhob", "rp", "nphi"]

In [ ]:
acumulada_key_well = 14.096
acumulada_well_target = 0.231

## Machine Learning

### Dolomite

In [5]:
# Definimos el target de interes
target = ["dolomite_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.024/0.038


In [6]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df = predict_well.copy()
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Ilite

In [7]:
# Definimos las columnas de interes
target = ["illite_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.004/0.011


In [8]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]

predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Calcite

In [9]:
# Definimos el target de interes
target = ["calcite_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.010/0.025


In [10]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### SW

In [11]:
# Definimos el target de interes
target = ["sw_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="squared_error", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.031/0.075


In [12]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Bound Water

In [13]:
# Definimos el target de interes
target = ["bound water_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.000/0.001


In [14]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Uoil

In [15]:
# Definimos el target de interes
target = ["uoil_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.001/0.003


In [16]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Uwater

In [17]:
# Definimos el target de interes
target = ["uwater_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.001/0.003


In [18]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### PHIT

In [19]:
# Definimos el target de interes
target = ["por_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.001/0.002


In [20]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### PHIE

In [21]:
# Definimos el target de interes
target = ["pore_qe"]

# Creamos un DataFrame con la informacion necesario para el modelo de Machine Learning
key_well_df_ml = key_well[features + target + key_well_tops].copy()
key_well_df_ml.dropna(inplace=True)

# Definimos una variable con los features con los que entrenaremos nuestros modelo de ML y con la variable a predecir
X = key_well_df_ml[features + key_well_tops]
y = key_well_df_ml[target]

# Separamos los valores en set de entramiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Convertimos en un array 1D y_train
y_train = y_train.values.ravel()

# Definimos el modelo a emplear
rf_regressor = RandomForestRegressor(bootstrap=True, n_jobs=-1, criterion="poisson", random_state=50)

# Ajustamos nuestro modelo a nuestros datos de entrenamiento
rf_regressor.fit(X_train, y_train)

# Evaluamos nuestro modelo
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

rf_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rf_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f'Random forest train/test RMSE: {rf_train: .3f}/{rf_test:.3f}')

RandomForestRegressor(criterion='poisson', n_jobs=-1, random_state=50)

Random forest train/test RMSE:  0.001/0.003


In [22]:
# Creamos el DataFrame predict_well con la informacion del pozo de interes
predict_well = cpe_df[cpe_df["wellname"] == well_target].copy()
predict_well = predict_well[["wellname", "md"] + features + key_well_tops]
predict_well.dropna(inplace=True)

# Creamos un DataFrame con la informacion para predecir
predict_well_ml = predict_well[features + key_well_tops].copy()

# Aplicamos el modelo para generar el registro sintetico
y_pred_well_log = rf_regressor.predict(predict_well_ml)

# Creamos la columna del registro sintetico
predict_well[target[0] +"_syn"] = y_pred_well_log

# Guardamos la curva en el dataset del pozo objetivo
qe_df[target[0] +"_syn"] = predict_well[target[0] +"_syn"]

### Corregimos los volumenes

In [23]:
# Cambiamos los valores de dolomite_qe_syn a 0 donde cretacico sea True
qe_df.loc[qe_df['cretacico'], 'dolomite_qe_syn'] = 0

# Creamos una columa con la suma de todos las variables volumetricas
qe_df["total_vol"] = qe_df["illite_qe_syn"] + qe_df["calcite_qe_syn"] + qe_df["dolomite_qe_syn"] + qe_df["bound water_qe_syn"] + qe_df["uoil_qe_syn"] + qe_df["uwater_qe_syn"]

# Calculamos la diferencia necesaria para alcanzar la unidad
diferencia = 1 - qe_df["total_vol"]

# Sumamos la diferencia a calcite y creamos una columna calcite_qe_corregido
qe_df["calcite_qe_corregido"] = qe_df["calcite_qe_syn"] + diferencia

# Actualizar los valores de dolomite_qe_syn a 0 donde cretacico sea True
qe_df.loc[qe_df['cretacico'], 'dolomite_qe_syn'] = 0

# Creamos una columna con la suma de todas las variables volumetricas corregidas
qe_df["total_vol_corregido"] = qe_df["illite_qe_syn"] + qe_df["calcite_qe_corregido"] + qe_df["dolomite_qe_syn"] + qe_df["bound water_qe_syn"] + qe_df["uoil_qe_syn"] + qe_df["uwater_qe_syn"]


qe_df[["total_vol","total_vol_corregido"]].describe()

,total_vol,total_vol_corregido
count,19376.000000,1.937600e+04
mean,0.883536,1.000000e+00
std,0.097544,5.210766e-17
min,0.584989,1.000000e+00
25%,0.842451,1.000000e+00
50%,0.883143,1.000000e+00
75%,0.890814,1.000000e+00
max,1.179554,1.000000e+00


## Guardamos el modelo de Machine Learning

In [24]:
# definimos las unidades para cada columna
units_row = {
    "wellname": " ",
    "md": "m",
    "illite_qe_syn": "v/v",
    "calcite_qe_corregido": "v/v",
    "dolomite_qe_syn": "v/v",
    "bound water_qe_syn": "v/v",
    "uoil_qe_syn": "v/v",
    "uwater_qe_syn": "v/v",
    "por_qe_syn": "v/v",
    "pore_qe_syn": "v/v",
    "sw_qe_syn":"v/v",
}

# Agregamos la fila con las unidades al DataFrame
qe_df = pd.concat([pd.DataFrame([units_row]), qe_df], ignore_index=True)

# Guardamos el registro sintetico en formato csv
synthetic_file_name = f"{well_target}_qelan_syn - calcite corregido.csv"
path_to_save = os.path.join(notebook_dir, "..", "data", "processed", synthetic_file_name)
qe_df[["wellname", "md", "illite_qe_syn", "calcite_qe_corregido", "dolomite_qe_syn", "bound water_qe_syn", "uoil_qe_syn", "uwater_qe_syn", "por_qe_syn", "pore_qe_syn", "sw_qe_syn"]].to_csv(path_to_save, index=False)